<a href="https://colab.research.google.com/github/hwangho-kim/Utility-OAC/blob/main/%EB%8C%80%ED%99%94%ED%98%95_%EC%8A%B5%EA%B3%B5%EA%B8%B0%EC%84%A0%EB%8F%84_(%EC%8A%AC%EB%9D%BC%EC%9D%B4%EB%8D%94_%EC%98%A4%EB%A5%98_%EC%88%98%EC%A0%95%2C_%EA%B2%BD%EB%A1%9C_%EC%BB%AC%EB%9F%BC%EB%AA%85_%EB%B3%80%EA%B2%BD).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import plotly.graph_objects as go
import numpy as np
import pandas as pd
import io # 문자열로부터 CSV를 읽기 위한 모듈
import os # 실제 파일 시스템 상호작용을 위한 모듈
import glob # 파일 패턴 매칭을 위한 모듈
import traceback # 예외 발생 시 상세 정보 출력을 위함

# 1. 기본 상수 정의
C_PA = 1.006  # 건공기의 정압비열 (kJ/kg°C)
C_PW = 1.86   # 수증기의 정압비열 (kJ/kg°C)
H_FG0 = 2501  # 0°C에서의 물의 증발잠열 (kJ/kg)
P_ATM = 101.325 # 표준 대기압 (kPa)

# 2. 습공기 상태량 계산 함수 - 변경 없음 (v6와 동일)
def get_saturation_vapor_pressure_kPa(temp_c):
    if pd.isna(temp_c) or temp_c < -60: return np.nan
    return 0.61094 * np.exp((17.625 * temp_c) / (temp_c + 243.04))

def get_actual_vapor_pressure_kPa(temp_c, rh_percent):
    if pd.isna(temp_c) or pd.isna(rh_percent): return np.nan
    p_ws = get_saturation_vapor_pressure_kPa(temp_c)
    if pd.isna(p_ws): return np.nan
    return (rh_percent / 100.0) * p_ws

def get_humidity_ratio_kg_kg(temp_c, rh_percent, p_atm_kPa=P_ATM):
    if pd.isna(temp_c) or pd.isna(rh_percent): return np.nan
    p_w = get_actual_vapor_pressure_kPa(temp_c, rh_percent)
    if pd.isna(p_w): return np.nan
    denominator = p_atm_kPa - p_w
    if denominator <= 1e-9:
        if abs(p_w - get_saturation_vapor_pressure_kPa(temp_c)) < 1e-6 and rh_percent > 99.99:
             p_ws_check = get_saturation_vapor_pressure_kPa(temp_c)
             if pd.isna(p_ws_check): return np.nan
             p_w_adjusted = p_ws_check * 0.99999
             denominator_adjusted = p_atm_kPa - p_w_adjusted
             if denominator_adjusted <= 1e-9: return float('inf')
             return 0.621945 * p_w_adjusted / denominator_adjusted
        else:
            return float('inf')
    return 0.621945 * p_w / denominator

def get_enthalpy_kJ_kg(temp_c, rh_percent, p_atm_kPa=P_ATM):
    if pd.isna(temp_c) or pd.isna(rh_percent): return np.nan
    w = get_humidity_ratio_kg_kg(temp_c, rh_percent, p_atm_kPa)
    if pd.isna(w) or w == float('inf'): return np.nan
    return C_PA * temp_c + w * (H_FG0 + C_PW * temp_c)

def get_temp_from_enthalpy_humidity_ratio(h_kJ_kg, w_kg_kg):
    if pd.isna(h_kJ_kg) or pd.isna(w_kg_kg): return np.nan
    denominator = C_PA + w_kg_kg * C_PW
    if abs(denominator) < 1e-9: return float('nan')
    return (h_kJ_kg - w_kg_kg * H_FG0) / denominator

# 3. Plotly용 습공기선도 배경 그리기 함수 - 변경 없음 (v6와 동일)
def add_psychrometric_background_to_fig(fig, temp_range_c, w_range_kg_kg):
    temps_c_plot = np.linspace(temp_range_c[0], temp_range_c[1], 100)
    w_saturated = np.array([get_humidity_ratio_kg_kg(t, 100) for t in temps_c_plot])
    valid_sat_indices = ~np.isnan(w_saturated) & ~np.isinf(w_saturated) & (w_saturated <= w_range_kg_kg[1])
    fig.add_trace(go.Scatter(
        x=temps_c_plot[valid_sat_indices], y=w_saturated[valid_sat_indices],
        mode='lines', name='상대습도 100%', line=dict(color='black', width=1.5),
        hoverinfo='skip', showlegend=True, legendgroup="background" # 범례 그룹화
    ))
    rh_curves_to_plot = [20, 40, 60, 80]
    for rh_val in rh_curves_to_plot:
        w_rh_plot = np.array([get_humidity_ratio_kg_kg(t, rh_val) for t in temps_c_plot])
        valid_rh_indices = ~np.isnan(w_rh_plot) & ~np.isinf(w_rh_plot) & (w_rh_plot <= w_range_kg_kg[1]) & (w_rh_plot >= w_range_kg_kg[0])
        if np.any(valid_rh_indices):
            fig.add_trace(go.Scatter(
                x=temps_c_plot[valid_rh_indices], y=w_rh_plot[valid_rh_indices],
                mode='lines', name=f'상대습도 {rh_val}%', line=dict(color='blue', width=0.8, dash='dash'),
                hoverinfo='skip', showlegend=True, legendgroup="background"
            ))
    h_min_approx = get_enthalpy_kJ_kg(temp_range_c[0], 20 if temp_range_c[0] > 0 else 80)
    h_max_approx = get_enthalpy_kJ_kg(temp_range_c[1], 100)
    if pd.isna(h_min_approx) or h_min_approx == float('inf'): h_min_approx = 0
    if pd.isna(h_max_approx) or h_max_approx == float('inf'):
        h_max_approx = get_enthalpy_kJ_kg(temp_range_c[1], 80, P_ATM)
        if pd.isna(h_max_approx) or h_max_approx == float('inf'): h_max_approx = 120
    enthalpy_lines_values = np.unique(np.linspace(round(h_min_approx / 10) * 10, round(h_max_approx / 10) * 10, 10).astype(int))
    enthalpy_lines_values = [h for h in enthalpy_lines_values if not pd.isna(h) and h != float('inf') and h != float('-inf')]
    for h_val in enthalpy_lines_values:
        w_for_h_line = np.linspace(w_range_kg_kg[0], w_range_kg_kg[1], 50)
        temps_for_h_line = np.array([get_temp_from_enthalpy_humidity_ratio(h_val, w) for w in w_for_h_line])
        valid_h_points_indices = ~np.isnan(temps_for_h_line) & \
                                 (temps_for_h_line >= temp_range_c[0]) & \
                                 (temps_for_h_line <= temp_range_c[1]) & \
                                 (w_for_h_line >= w_range_kg_kg[0]) & \
                                 (w_for_h_line <= w_range_kg_kg[1])
        if np.sum(valid_h_points_indices) > 1:
            temps_plot_h = temps_for_h_line[valid_h_points_indices]
            w_plot_h = w_for_h_line[valid_h_points_indices]
            fig.add_trace(go.Scatter(
                x=temps_plot_h, y=w_plot_h, mode='lines',
                name=f'엔탈피={h_val}kJ/kg', line=dict(color='green', width=0.8, dash='dot'),
                hoverinfo='skip', showlegend=False, legendgroup="background"
            ))
            if len(temps_plot_h) > 0:
                text_x, text_y = temps_plot_h[0], w_plot_h[0]
                if temp_range_c[0] <= text_x <= temp_range_c[1] and w_range_kg_kg[0] <= text_y <= w_range_kg_kg[1]:
                    fig.add_annotation(x=text_x, y=text_y, text=f"{h_val}", showarrow=False,
                                       font=dict(color="green", size=9), xanchor="right", yanchor="bottom", textangle=-30)

# --- 파일 및 컬럼 처리 함수 - 변경 없음 (v6와 동일) ---
def find_oac_csv_files_simulated(virtual_file_system):
    print("시뮬레이션된 파일 검색 중...")
    oac_files = {name: content for name, content in virtual_file_system.items() if "oac" in name.lower() and name.endswith(".csv")}
    if not oac_files:
        print("시뮬레이션된 'OAC' CSV 파일을 찾을 수 없습니다.")
        return {}
    return oac_files

def select_csv_file_simulated(oac_files_dict):
    if not oac_files_dict:
        return None, None
    print("\n사용 가능한 'OAC' CSV 파일 (시뮬레이션):")
    file_list = list(oac_files_dict.keys())
    for i, filename in enumerate(file_list):
        print(f"  {i+1}. {filename}")
    while True:
        try:
            choice = int(input(f"파일 번호를 선택하세요 (1-{len(file_list)}): "))
            if 1 <= choice <= len(file_list):
                selected_filename = file_list[choice-1]
                return selected_filename, oac_files_dict[selected_filename]
            else:
                print("잘못된 선택입니다. 목록에 있는 번호를 입력하세요.")
        except ValueError:
            print("잘못된 입력입니다. 숫자를 입력하세요.")

STAGE_MAPPING_DEFINITIONS = [
    {'id': 'PHC', 'display_name': "1. PH/C (예열)", 'stage_keywords': ["PH/C", "PREHEAT", "예열"], 'temp_keyword': "온도", 'rh_keyword': "습도", 'color': '#ff7f0e'},
    {'id': 'PCC', 'display_name': "2. PC/C (예냉)", 'stage_keywords': ["PC/C", "PRECOOL", "예냉"], 'temp_keyword': "온도", 'rh_keyword': "습도", 'color': '#2ca02c'},
    {'id': 'CC',  'display_name': "3. C/C (냉각)", 'stage_keywords': ["C/C", "COOLINGCOIL", "냉각코일", "COOLING", "COOL"], 'temp_keyword': "온도", 'rh_keyword': "습도", 'color': '#d62728'},
    {'id': 'HC',  'display_name': "4. H/C (가열/재열)", 'stage_keywords': ["H/C", "HEATINGCOIL", "재열코일", "승온", "HEATING", "HEAT", "REHEAT"], 'temp_keyword': "온도", 'rh_keyword': "습도", 'color': '#9467bd'}
]

def map_columns_from_csv(df_columns):
    mapped_stages_config = []
    print("\n컬럼 매핑 시도 중...")
    available_columns = list(df_columns)
    for stage_def in STAGE_MAPPING_DEFINITIONS:
        found_temp_col = None
        found_rh_col = None
        for col_name in available_columns:
            col_lower = col_name.lower()
            if any(sk.lower() in col_lower for sk in stage_def['stage_keywords']) and \
               stage_def['temp_keyword'].lower() in col_lower:
                if found_temp_col is None:
                    found_temp_col = col_name
        for col_name in available_columns:
            if col_name == found_temp_col:
                 continue
            col_lower = col_name.lower()
            if any(sk.lower() in col_lower for sk in stage_def['stage_keywords']) and \
               stage_def['rh_keyword'].lower() in col_lower:
                if found_rh_col is None:
                    found_rh_col = col_name
        if found_temp_col and found_rh_col:
            mapped_stages_config.append({
                'name': stage_def['display_name'],
                'temp_col': found_temp_col,
                'rh_col': found_rh_col,
                'color': stage_def['color']
            })
            print(f"  스테이지 매핑 성공: {stage_def['display_name']}")
            print(f"    온도 컬럼: '{found_temp_col}'")
            print(f"    습도 컬럼: '{found_rh_col}'")
            if found_temp_col in available_columns:
                available_columns.remove(found_temp_col)
            if found_rh_col in available_columns:
                available_columns.remove(found_rh_col)
        elif found_temp_col or found_rh_col :
             print(f"  스테이지 부분 매핑: {stage_def['display_name']}. 온도: {found_temp_col}, 습도: {found_rh_col}. 차트에 미포함.")
    if not mapped_stages_config:
        print("경고: CSV 컬럼에서 매핑할 수 있는 스테이지가 없습니다.")
    return mapped_stages_config

# 4. 대화형 차트 생성 함수
def create_interactive_psychrometric_chart_plotly(df, mapped_stages_config, selected_filename):
    if df.empty:
        print("데이터프레임이 비어있어 차트를 생성할 수 없습니다.")
        return
    if not mapped_stages_config:
        print("매핑된 스테이지 설정이 없어 차트를 생성할 수 없습니다.")
        return

    all_temps_for_bounds = []
    all_ws_for_bounds = []
    for index, row in df.iterrows():
        for stage in mapped_stages_config:
            try:
                temp = row[stage['temp_col']]
                rh = row[stage['rh_col']]
                if pd.isna(temp) or pd.isna(rh): continue
                all_temps_for_bounds.append(temp)
                w = get_humidity_ratio_kg_kg(temp, rh)
                if not pd.isna(w) and w != float('inf') and w < 0.1:
                    all_ws_for_bounds.append(w)
            except KeyError as e:
                continue

    if not all_temps_for_bounds or not all_ws_for_bounds:
        print("차트 범위 계산을 위한 유효한 데이터 포인트가 없습니다. 컬럼 매핑 및 데이터를 확인하세요.")
        min_temp_data, max_temp_data = 0, 40
        min_w_data, max_w_data = 0, 0.025
    else:
        min_temp_data, max_temp_data = min(all_temps_for_bounds), max(all_temps_for_bounds)
        min_w_data, max_w_data = min(all_ws_for_bounds), max(all_ws_for_bounds)

    temp_margin = (max_temp_data - min_temp_data) * 0.1 if (max_temp_data - min_temp_data) > 0 else 5
    w_margin = (max_w_data - min_w_data) * 0.1 if (max_w_data - min_w_data) > 0 else 0.002
    chart_min_temp = min_temp_data - temp_margin
    chart_max_temp = max_temp_data + temp_margin * 1.5
    chart_min_w = 0
    chart_max_w = max_w_data + w_margin * 2.0
    w_sat_at_max_temp_chart = get_humidity_ratio_kg_kg(chart_max_temp, 100)
    if not pd.isna(w_sat_at_max_temp_chart) and w_sat_at_max_temp_chart != float('inf'):
        chart_max_w = max(chart_max_w, w_sat_at_max_temp_chart * 1.05)
    chart_max_w = min(chart_max_w, 0.035)

    fig = go.Figure()
    add_psychrometric_background_to_fig(fig, (chart_min_temp, chart_max_temp), (chart_min_w, chart_max_w))

    # 동적 트레이스에 UID 추가하여 프레임 업데이트 시 명확히 식별
    fig.add_trace(go.Scatter(
        uid="process_points_trace", # UID 추가
        x=[], y=[], mode='markers', marker=dict(size=10), name="공조 과정 지점",
        text=[], hovertemplate="<b>%{text}</b><br>온도: %{x:.1f}°C<br>절대습도: %{y:.4f} kg/kg<br>상대습도: %{customdata[0]:.0f}%<br>엔탈피: %{customdata[1]:.1f} kJ/kg<extra></extra>",
        customdata=[], showlegend=False
    ))
    fig.add_trace(go.Scatter(
        uid="process_path_trace", # UID 추가
        x=[], y=[], mode='lines', line=dict(color='lightgrey', width=2), name="공조 과정 경로", # 색상 변경: lightgrey
        hoverinfo='skip', showlegend=False
    ))

    frames = []
    print("차트 프레임 생성 중... 데이터 양에 따라 시간이 소요될 수 있습니다.")
    for idx, row_data in df.iterrows():
        timestamp_str = row_data['Timestamp'].strftime('%Y-%m-%d %H:%M:%S') # 프레임 이름용
        frame_points_x = []
        frame_points_y = []
        frame_points_text = []
        frame_points_customdata = []
        frame_points_colors = []
        current_annotations = [] # 각 프레임마다 주석 리스트 초기화

        for i, stage_conf in enumerate(mapped_stages_config):
            temp_c, rh_p = np.nan, np.nan
            try:
                temp_c = row_data[stage_conf['temp_col']]
                rh_p = row_data[stage_conf['rh_col']]
            except KeyError:
                pass

            if pd.isna(temp_c) or pd.isna(rh_p):
                w, h = np.nan, np.nan
            else:
                w = get_humidity_ratio_kg_kg(temp_c, rh_p)
                h = get_enthalpy_kJ_kg(temp_c, rh_p)

            if not (pd.isna(w) or pd.isna(h) or w == float('inf') or h == float('inf') or w >= chart_max_w * 1.5) :
                frame_points_x.append(temp_c)
                frame_points_y.append(w)
                frame_points_text.append(stage_conf['name'])
                frame_points_customdata.append([rh_p, h])
                frame_points_colors.append(stage_conf['color'])
                current_annotations.append(dict(
                    x=temp_c, y=w,
                    text=f"<b>{stage_conf['name']}</b><br>{temp_c:.1f}°C, {rh_p:.0f}%<br>h={h:.1f}",
                    showarrow=True, arrowhead=1, arrowwidth=1, arrowcolor=stage_conf['color'],
                    ax=20, ay=-30, font=dict(size=9, color='black'),
                    bordercolor=stage_conf['color'], borderwidth=1, bgcolor="rgba(255,255,255,0.8)" ))
            else:
                frame_points_x.append(None)
                frame_points_y.append(None)
                frame_points_text.append(stage_conf['name'] + " (값 없음)")
                frame_points_customdata.append([np.nan, np.nan])
                frame_points_colors.append('rgba(128,128,128,0.5)')

        frames.append(go.Frame(
            name=timestamp_str, # 프레임 식별자
            data=[ # 이 프레임에서 업데이트할 트레이스 데이터 (UID 사용)
                go.Scatter(uid="process_points_trace", x=frame_points_x, y=frame_points_y, text=frame_points_text,
                           customdata=np.array(frame_points_customdata).T if frame_points_customdata and any(cd is not None for cd_list in frame_points_customdata for cd in cd_list) else np.array([]).reshape(2,0),
                           marker=dict(color=frame_points_colors, size=10, line=dict(width=1, color='DarkSlateGrey'))),
                go.Scatter(uid="process_path_trace", x=frame_points_x, y=frame_points_y, line=dict(color='lightgrey', width=2)) # 경로선 색상
            ],
            layout=go.Layout( # 이 프레임에서 업데이트할 레이아웃 속성
                annotations=current_annotations, # 현재 프레임의 주석으로 완전히 교체
                title_text=f"대화형 습공기선도: {selected_filename} - {timestamp_str}"
            )
        ))
    print("프레임 생성 완료.")

    fig.frames = frames
    sliders = [dict(active=0, currentvalue={"prefix": "시간: ", "font": {"size": 14}}, pad={"t": 50, "b":10}, steps=[])]
    for i, timestamp_obj in enumerate(df['Timestamp']):
        if isinstance(timestamp_obj, str):
            try:
                current_ts = pd.to_datetime(timestamp_obj)
            except ValueError:
                continue
        else:
            current_ts = timestamp_obj

        ts_str_frame_name = current_ts.strftime('%Y-%m-%d %H:%M:%S')
        ts_label_slider = current_ts.strftime('%m-%d %H:%M')

        slider_step = dict(label=ts_label_slider, method="animate",
                           args=[[ts_str_frame_name], dict(mode="immediate", frame=dict(duration=30, redraw=True), # duration 더 줄임
                                                transition=dict(duration=0))]) # transition duration 0
        sliders[0]['steps'].append(slider_step)

    initial_timestamp_str = df['Timestamp'].iloc[0].strftime('%Y-%m-%d %H:%M:%S')

    fig.update_layout(
        title_text=f"대화형 습공기선도: {selected_filename} - {initial_timestamp_str}",
        width=1200,
        height=800,
        xaxis=dict(title='건구온도 (°C)', range=[chart_min_temp, chart_max_temp], autorange=False), # autorange False
        yaxis=dict(title='절대습도 (kg_water / kg_dry_air)', range=[chart_min_w, chart_max_w], autorange=False), # autorange False
        sliders=sliders,
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1, traceorder="normal"),
        margin=dict(l=60, r=60, b=120, t=100),
        hovermode="closest"
    )

    # 초기 프레임 데이터 및 레이아웃 명시적 설정
    if frames:
        first_frame = frames[0]
        # UID를 사용하여 트레이스 업데이트 (더 안정적)
        for trace_update in first_frame.data:
            fig.update_traces(selector=dict(uid=trace_update.uid), **trace_update)

        if hasattr(first_frame.layout, 'annotations'):
             fig.layout.annotations = first_frame.layout.annotations
        else:
             fig.layout.annotations = [] # 주석이 없으면 빈 리스트로 초기화
    else:
        print("경고: 생성된 프레임이 없습니다. 슬라이더가 제대로 작동하지 않을 수 있습니다.")
        fig.layout.annotations = [] # 프레임 없으면 주석 없음

    print("차트 표시 중...")
    fig.show()

# --- 더미 CSV 데이터 (컬럼명 다양화) - 변경 없음 ---
dummy_csv_data_ahu1_oac = """Timestamp,AHU01_PH/C_온도_입구,AHU01_PH/C_습도_입구,AHU01_PC/C_Zone1_온도,AHU01_PC/C_Zone1_습도,AHU01_CoolingCoil_Air_온도_Avg,AHU01_CoolingCoil_Air_습도_Avg,AHU01_HC_승온후_온도값,AHU01_HC_승온후_습도값
2023-07-15 10:00:00,30.0,65,27.0,75,16.0,92,21.0,68
2023-07-15 10:10:00,30.2,66,26.5,76,14.0,96,23.0,58
2023-07-15 10:20:00,30.5,64,26.0,77,12.0,99,25.0,48
2023-07-15 11:00:00,31.0,65,26.0,77,12.5,98,25.5,47
2023-07-15 11:10:00,31.2,66,25.5,78,13.0,97,26.0,46
2023-07-15 11:59:59,31.5,66.5,25.0,78.5,13.5,96.5,26.5,45.5
2023-07-15 12:00:00,30.0,70,28.0,80,18.0,90,20.0,70
"""
dummy_csv_data_ahu2_oac_control = """DATETIME,ZoneA_PREHEAT_온도,ZoneA_PREHEAT_습도,ZoneA_PRECOOL_온도,ZoneA_PRECOOL_습도,MainUnit_C/C_온도_출구,MainUnit_C/C_습도_출구,PostZone_HEAT_온도,PostZone_HEAT_습도
2023-07-16 14:00:00,28.0,70,25.0,80,17.0,88,20.0,65
2023-07-16 14:10:00,28.2,72,24.5,81,16.0,93,22.5,55
2023-07-16 14:20:00,28.5,71,24.0,82,15.0,97,24.5,45
2023-07-16 15:00:00,29.0,70,24.0,82,15.5,96,25.0,44
"""
dummy_csv_data_no_oac = """Timestamp,Sensor_Temp,Sensor_RH
2023-07-17 09:00:00,22,55
"""
VIRTUAL_FILE_SYSTEM = {
    "data_ahu1_oac.csv": dummy_csv_data_ahu1_oac,
    "data_ahu2_oac_control_DATETIME.csv": dummy_csv_data_ahu2_oac_control, # DATETIME 컬럼명 테스트용
    "other_sensor_data.csv": dummy_csv_data_no_oac,
    "report_oac_summary.txt": "이것은 CSV가 아닙니다."
}

# --- 실행 부분 ---
if __name__ == '__main__':
    use_real_files = input("실제 파일 시스템을 사용하시겠습니까? (y/n, 기본값 n): ").lower() == 'y'
    selected_filename = None
    df_to_process = None
    internal_timestamp_col = 'Timestamp' # 내부적으로 사용할 표준 타임스탬프 컬럼명

    if use_real_files:
        DATA_DIRECTORY = "./24Y/" # 고정된 디렉토리 경로
        print(f"지정된 디렉토리: {DATA_DIRECTORY}")
        if not os.path.isdir(DATA_DIRECTORY):
            print(f"오류: 디렉토리를 찾을 수 없습니다: {DATA_DIRECTORY}")
            print(f"현재 작업 디렉토리({os.getcwd()}) 하위에 '24Y' 폴더를 생성하고 CSV 파일을 넣어주세요.")
            exit()

        actual_oac_files = []
        print(f"\n'{DATA_DIRECTORY}' 디렉토리에서 'OAC' 포함 CSV 파일 검색 중...")
        for filename_in_dir in os.listdir(DATA_DIRECTORY):
            if "oac" in filename_in_dir.lower() and filename_in_dir.endswith(".csv"):
                actual_oac_files.append(os.path.join(DATA_DIRECTORY, filename_in_dir))

        if not actual_oac_files:
            print(f"디렉토리에서 'OAC' 포함 CSV 파일을 찾을 수 없습니다: {DATA_DIRECTORY}")
            exit()

        print("\n사용 가능한 'OAC' CSV 파일:")
        for i, filepath in enumerate(actual_oac_files):
            print(f"  {i+1}. {os.path.basename(filepath)}")

        selected_filepath = None
        while True:
            try:
                choice = int(input(f"파일 번호를 선택하세요 (1-{len(actual_oac_files)}): "))
                if 1 <= choice <= len(actual_oac_files):
                    selected_filepath = actual_oac_files[choice-1]
                    selected_filename = os.path.basename(selected_filepath)
                    break
                else:
                    print("잘못된 선택입니다. 목록에 있는 번호를 입력하세요.")
            except ValueError:
                print("잘못된 입력입니다. 숫자를 입력하세요.")

        if selected_filepath:
            print(f"\n실제 파일에서 데이터 로딩 중: {selected_filename}")
            print("참고: 파일 크기가 매우 큰 경우 초기 로딩에 다소 시간이 소요될 수 있습니다...")
            try:
                df_to_process = pd.read_csv(selected_filepath)
            except Exception as e:
                print(f"파일 로딩 중 오류 발생: {e}")
                exit()
    else:
        available_oac_files = find_oac_csv_files_simulated(VIRTUAL_FILE_SYSTEM)
        selected_filename, selected_csv_content = select_csv_file_simulated(available_oac_files)
        if selected_csv_content:
            print(f"\n선택된 시뮬레이션 파일에서 데이터 로딩 중: {selected_filename}")
            try:
                df_to_process = pd.read_csv(io.StringIO(selected_csv_content))
            except Exception as e:
                print(f"시뮬레이션된 파일 처리 중 오류 발생: {e}")
                exit()

    if df_to_process is not None and selected_filename:
        try:
            # 'DATETIME' 또는 'Timestamp' 컬럼을 내부 표준 'Timestamp'로 통일
            if 'DATETIME' in df_to_process.columns:
                df_to_process.rename(columns={'DATETIME': internal_timestamp_col}, inplace=True)
            elif internal_timestamp_col not in df_to_process.columns: # 'Timestamp'도 없는 경우
                 raise ValueError(f"CSV 파일에 '{internal_timestamp_col}' 또는 'DATETIME' 컬럼이 반드시 포함되어야 합니다.")

            df_to_process[internal_timestamp_col] = pd.to_datetime(df_to_process[internal_timestamp_col])

            print(f"원본 데이터 포인트 수: {len(df_to_process)}")
            df_to_process.set_index(internal_timestamp_col, inplace=True)

            numeric_cols = df_to_process.select_dtypes(include=np.number).columns
            df_resampled = pd.DataFrame()

            if not numeric_cols.empty:
                 valid_numeric_cols = [col for col in numeric_cols if col in df_to_process.columns]
                 if valid_numeric_cols:
                    print("데이터를 1시간 단위 첫 번째 값으로 리샘플링합니다...")
                    df_resampled = df_to_process[valid_numeric_cols].resample('1H').first()
                    df_resampled.dropna(how='all', inplace=True)
                 else:
                    print("경고: 유효한 숫자형 컬럼이 없어 리샘플링을 위한 데이터가 없습니다. 원본 데이터를 사용합니다.")
                    df_resampled = df_to_process.copy()
            else:
                 print("경고: 데이터에 숫자형 컬럼이 없어 리샘플링을 건너<0xEB><0><0x8A>니다. 원본 데이터를 사용합니다.")
                 df_resampled = df_to_process.copy()

            if not isinstance(df_resampled.index, pd.DatetimeIndex):
                if internal_timestamp_col in df_resampled.columns:
                    try:
                        df_resampled[internal_timestamp_col] = pd.to_datetime(df_resampled[internal_timestamp_col])
                        df_resampled.set_index(internal_timestamp_col, inplace=True)
                        print("Timestamp 컬럼을 인덱스로 설정했습니다.")
                    except Exception as e:
                        print(f"Timestamp 컬럼을 인덱스로 설정하는 중 오류 발생: {e}. 차트 생성에 문제가 있을 수 있습니다.")
                else:
                    print("경고: Timestamp 인덱스 또는 컬럼을 찾을 수 없습니다. 원본 데이터 구조를 확인해주세요.")

            df_resampled.reset_index(inplace=True)

            if internal_timestamp_col not in df_resampled.columns and not df_resampled.empty:
                # 만약 reset_index 후에도 'Timestamp'가 없다면, 인덱스 이름이 달랐을 수 있음.
                # 이 경우, index.name을 확인하고 해당 이름으로 컬럼을 가져오거나,
                # df_resampled = df_resampled.rename(columns={df_resampled.index.name: internal_timestamp_col}) 시도 가능
                print(f"오류: 리샘플링 후 '{internal_timestamp_col}' 컬럼이 사라졌습니다. 원본 데이터의 컬럼명을 확인해주세요.")

            print(f"리샘플링 후 데이터 포인트 수: {len(df_resampled)}")

            if df_resampled.empty or internal_timestamp_col not in df_resampled.columns:
                print(f"리샘플링 후 데이터가 없거나 '{internal_timestamp_col}' 컬럼이 없습니다. 차트를 생성할 수 없습니다.")
            else:
                # Timestamp 컬럼명을 슬라이더 및 프레임 생성 함수에 전달되는 df에서 일관되게 'Timestamp'로 사용
                if internal_timestamp_col != 'Timestamp' and 'Timestamp' not in df_resampled.columns: # 내부 표준명과 다르고, 'Timestamp'가 없다면
                    df_resampled.rename(columns={internal_timestamp_col: 'Timestamp'}, inplace=True)

                if len(df_resampled) > 2000:
                    print(f"주의: 리샘플링 후에도 데이터 포인트(프레임 수)가 {len(df_resampled)}개로 매우 많습니다. ")
                    print("차트 로딩 및 슬라이더 반응에 많은 시간이 소요될 수 있습니다.")

                mapped_stages = map_columns_from_csv(df_resampled.columns)
                if mapped_stages:
                    create_interactive_psychrometric_chart_plotly(df_resampled, mapped_stages, selected_filename)
                else:
                    print("\n컬럼 매핑 문제로 차트 생성을 진행할 수 없습니다.")

        except Exception as e:
            print(f"파일 {selected_filename} 처리 중 오류 발생: {e}")
            traceback.print_exc()
    elif not selected_filename :
        print("파일이 선택되지 않았거나 시뮬레이션된 파일 내용이 없습니다. 프로그램을 종료합니다.")